# 02AO. Ao Court Vision Raw Data Scraper

Notebook will contain codes and functions for scraping the court vision raw data from the Roland Garros website.
Output is archived in json format.

In [3]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import re
import json
import itertools
import sys
sys._enablelegacywindowsfsencoding() #Deal with pandas problem with reading file with accents in file path i.e Alexis Sánchez, Victor Lindelöf 


headers = {'User-Agent': 
           'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/47.0.2526.106 Safari/537.36'} 

from ast import literal_eval

import base64
import cryptography.hazmat.backends
import cryptography.hazmat.primitives.ciphers
import cryptography.hazmat.primitives.ciphers.algorithms
import cryptography.hazmat.primitives.ciphers.modes
import cryptography.hazmat.primitives.padding

import sys
from time import sleep
import datetime

### Decrypting Utilities

In [4]:
def formatDate(t):
    #e = datetime.datetime.now().utcoffset().total_seconds() / 60       # ChatGPT suggestion but not needed
    #t = t + datetime.timedelta(minutes=e)                              # ChatGPT suggestion but not needed
    
    t_tstamp = datetime.datetime.utcfromtimestamp(t/1000)
    n = t_tstamp.day
    r = int(str(n if n >= 10 else "0" + str(n))[::-1])
    i = t_tstamp.year
    a = int(str(i)[::-1])
    o = np.base_repr(int(str(t), base=16), 36).lower() + np.base_repr((i + a) * (n + r), 24).lower()
    s = len(o)
    if s < 14:
        o += "0" * (14 - s)
    elif s > 14:
        o = o[:14]
    return "#" + o + "$"


In [5]:
def decode(data):
    e = formatDate(data['lastModified'])
    n = e.encode()
    r = e.upper().encode()
    cipher = cryptography.hazmat.primitives.ciphers.Cipher(
        cryptography.hazmat.primitives.ciphers.algorithms.AES(n),
        cryptography.hazmat.primitives.ciphers.modes.CBC(r),
        backend=cryptography.hazmat.backends.default_backend()
    )
    decryptor = cipher.decryptor()
    i = decryptor.update(base64.b64decode(data['response'])) + decryptor.finalize()
    unpadder = cryptography.hazmat.primitives.padding.PKCS7(128).unpadder()
    #return json.loads(unpadder.update(i) + unpadder.finalize().decode('utf-8'))
    return json.loads(i.decode("utf-8").replace(i.decode("utf-8")[-1],""))

## Get Match Data and URLs

In [6]:
import glob

In [7]:
matches = pd.read_csv("../data/grandslam-results/AO_results-all_processed_2023.csv") 

In [10]:
matches.head()

,date,actual_start_time,match_centre_link,uuid,match_id,team_substituted_footnote,team_substituted,id,order,promoted,...,player1_name,team2_player_uuid,player2_name,event_uuid,court_id,session,session_order,restricted_start_time,restricted_start_time_timestamp,activity_order
0,2023-01-16,00:14,https://ausopen.com/match/2023-katerina-siniak...,222446,WS116,NaN,False,222446,16,True,...,Katerina Siniakova,['2435e580-912e-4239-92c7-5778a17a7190'],Coco Gauff,4225c89e-7cfd-463f-a01d-4102ba2913f3,12,Day session,0,NaN,NaN,0
1,2023-01-16,01:50,https://ausopen.com/match/2023-yue-yuan-vs-mar...,222406,WS132,NaN,False,222406,32,False,...,Yue Yuan,['d5ff642d-4e0f-4e07-bcc9-1cffa2fe0dee'],Maria Sakkari,4225c89e-7cfd-463f-a01d-4102ba2913f3,12,Day session,0,NaN,NaN,1
2,2023-01-16,03:51,https://ausopen.com/match/2023-rafael-nadal-vs...,225666,MS101,NaN,False,225666,1,True,...,Rafael Nadal,['cd126eb3-67bf-4f1d-bd7c-c85a81b43afb'],Jack Draper,675d28ec-b177-46b2-959c-595f3ca862a0,12,Day session,0,14:30,1.673840e+09,2
3,2023-01-16,08:34,https://ausopen.com/match/2023-iga-swiatek-vs-...,222471,WS101,NaN,False,222471,1,True,...,Iga Swiatek,['a386d06b-811c-486b-96c6-45d54f560ae3'],Jule Niemeier,4225c89e-7cfd-463f-a01d-4102ba2913f3,12,Night session,1,NaN,NaN,0
4,2023-01-16,11:02,https://ausopen.com/match/2023-marcos-giron-vs...,225221,MS116,NaN,False,225221,16,False,...,Marcos Giron,['d36bef3a-e455-4180-bc58-b012c8c0b14a'],Daniil Medvedev,675d28ec-b177-46b2-959c-595f3ca862a0,12,Night session,1,NaN,NaN,1


In [26]:
matches_scp = matches[matches.match_id.str.contains("WS")]
matches_scp.tail(1)

,date,actual_start_time,match_centre_link,uuid,match_id,team_substituted_footnote,team_substituted,id,order,promoted,...,player1_name,team2_player_uuid,player2_name,event_uuid,court_id,session,session_order,restricted_start_time,restricted_start_time_timestamp,activity_order
655,2023-01-28,08:45,https://ausopen.com/match/2023-elena-rybakina-...,222196,WS701,NaN,False,222196,1,False,...,Elena Rybakina,['39a4ab8b-5ed7-4162-9a00-2b2c65cb63b8'],Aryna Sabalenka,4225c89e-7cfd-463f-a01d-4102ba2913f3,12,Night session,1,NaN,NaN,0


In [12]:
eventId = 580
year = 2023

In [27]:
# Loop through matches and scrape one by one 
for i in np.arange(0,len(matches_scp)):
    try:
        match_id = matches_scp.match_id.iloc[i]
        player1 = matches_scp.player1_name.iloc[i]
        player2 = matches_scp.player2_name.iloc[i]

        link = f"https://itp-ao-sls.infosys-platforms.com/prod/api/court-vision/year/{year}/eventId/{eventId}/matchId/{match_id}/pointId/0_0_0"
        
        # Get request and content from the given link and parse into HTML
        pageTree = requests.get(link, headers=headers)
        pageSoup = BeautifulSoup(pageTree.content, 'html.parser') 

        results_json = json.loads(str(pageSoup))

        # Decode Data
        raw_data = decode(results_json)

        # Match the formatting of player1/2 to that in the court-vision raw data's player data 
        # If player names match their respective indexes in the court-vision raw data, 
        # then we keep the player name order, otherwise we swap 
        # "Truncated Name" for player 1 (e.g. R. NADAL)
        player1_tname = player1.split(" ")[0][0]+". " + player1.split(" ")[1]
        player1_cv = raw_data['courtVisionData'][0]['a79']['a83'][0]['a85']

        if player1_tname == player1_cv:
            player1_cvfile = player1
            player2_cvfile = player2
        else:
            player1_cvfile = player2
            player2_cvfile = player1

        # Formatting
        player1_cvfile = player1_cvfile.replace(" ","-")
        player2_cvfile = player2_cvfile.replace(" ","-")

        # Format the "Round Name" to appear on file path
        round_n = matches_scp["round"].iloc[i]
        if round_n == "1st Round":
            round_short = "R128"
        elif round_n == "2nd Round":
            round_short = "R64"
        elif round_n == "3rd Round":
            round_short = "R32"
        elif round_n == "4th Round":
            round_short = "R16"
        elif round_n == "Quarterfinals" or round_n == "Quarter-Finals":
            round_short = "QF"
        elif round_n == "Semifinals" or round_n == "Semi-Finals":
            round_short = "SF"
        elif round_n == "Final" or round_n == "Finals":
            round_short = "F"

        # Output the decoded courtvision data into a json file
        with open(f"../data/court-vision/{eventId}_{round_short}_{player1_cvfile}-vs-{player2_cvfile}_{year}_{match_id}_court-vision.json", 'w') as fp:
            json.dump(raw_data, fp)

        sleeptime = np.random.uniform(3, 20)
        sleep(sleeptime)

    except:
        print(f"{i} Failed or no Data!")
        pass